In [ ]:
import json
import os

import openai
from openai_functools import FunctionsOrchestrator

In [ ]:
def retrieve_logs():
    pass

def query_db():
    pass

def query_api():
    pass

def query_vector_db():
    """TODO build docstrings"""
    return "spoof answer"

In [ ]:
orchestrator = FunctionsOrchestrator()
orchestrator.register_all([retrieve_logs, query_db, query_api, query_vector_db])

In [ ]:
def respond_and_call():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
        functions=orchestrator.create_function_descriptions(),
        function_call="auto",
    )

    # Call the function that is specified in the response
    response = orchestrator.call_function(response)

In [ ]:
 openai.api_key = os.environ["OPENAI_API_KEY"]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
        functions=orchestrator.create_function_descriptions(),
        function_call="auto",
    )
    # Call the function that is specified in the response
    print(orchestrator.call_function(response))

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": "What's the weather like in Boston tomorrow?"}
        ],
        functions=orchestrator.create_function_descriptions(),
        function_call="auto",
    )
    # Call the function that is specified in the response
    print(orchestrator.call_function(response))